In [15]:
!pip install -U tensorflow==2.15.0
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


In [31]:
import json

# Example data (a Python dictionary)
data = {
    "dataset": "/kaggle/input/mmmai-regist-data/MR-ART-Regist",
    "data_ids": {
      "/kaggle/input/mmmai-simulated-data/ds004795-download": "Motion_Simulated",
      "/kaggle/input/mmmai-regist-data/MR-ART-Regist": "Motion",
      "/kaggle/input/brats-motion-data/new_Brats_motion_data": "BraTS"
    },
    "loss_functions": [
      "ssim_loss",
      "perceptual_loss",
    ],
    "model_architecture": "wat_stacked_unets",
    "checkpoint_path": None,
    "epochs": 4,
    "batch_size": 10,
    "enable_SAP": True,
    "start_epoch": 0,
    
    "learning_rate": 0.001,
    "height": 256,
    "width": 256,
    "token":"2uuou6iTpP5tFnIjF3KZjJ4e5sm_58r6P5rmD9ohT8WYS24bP",
    "data_loader": {
      "split_ratio": [0.7, 0.2, 0.1],
      "view": "Axial",
      "crop": False,
      "split_json_path": None
    },
    "weights_path": "/kaggle/working/",
    "callbacks": {
      "csv_logger": {
        "enabled": True,
        "filename": "_Loss_Acc.csv"
      },
      "reduce_lr": {
        "enabled": True,
        "schedule_function": "exponential_lr"
      },
      "model_checkpoint": {
        "enabled": True,
        "filename_pattern": "wat_stacked_unets_epoch-{epoch:02d}_val_loss-{val_loss:.4f}.h5"
      }
    }
  }
  

# Specify the path where you want to save the JSON file
json_file_path = "/kaggle/working/config.json"

# Write the data to a JSON file
with open(json_file_path, "w") as json_file:
    json.dump(data, json_file, indent=4)  # `indent=4` makes the JSON file more readable

print(f"JSON file saved at: {json_file_path}")

JSON file saved at: /kaggle/working/config.json


In [17]:
!git clone https://github.com/mahmoud1yaser/mmmai_codebase

Cloning into 'mmmai_codebase'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 107 (delta 30), reused 98 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (107/107), 34.93 KiB | 2.18 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [18]:
%cd /kaggle/working/mmmai_codebase

/kaggle/working/mmmai_codebase


In [19]:
!git pull

Already up to date.


In [20]:
!python /kaggle/working/mmmai_codebase/src/training/trainer.py --config /kaggle/working/config.json

2025-03-27 21:10:27.966430: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-27 21:10:27.966502: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-27 21:10:27.968176: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
VGG16 Model Initialized
---------------------------------
Model Training ...
---------------------------------
42it [00:11,  7.92it/s]Wrong shape detected free shape = (4, 256, 256, 1) while motion shape = (4, 256, 256, 1)
43it [00:11,  3.61it/s]
Number of losses: 2
Weight (w_comb): [1.0, 0.16310966]
Bias (b_comb): [0.0, 0.02293662]
Epoch 1/4
I0000 00:00:17431098

In [21]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar xf ./ngrok-v3-stable-linux-amd64.tgz -C /usr/local/bin

--2025-03-27 21:39:10--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 75.2.60.68, 99.83.220.108, 35.71.179.82, ...
Connecting to bin.equinox.io (bin.equinox.io)|75.2.60.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9372184 (8.9M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz.1’

ngrok-v3-stable-lin 100%[===================>]   8.94M  4.91MB/s    in 1.8s    

2025-03-27 21:39:13 (4.91 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz.1’ saved [9372184/9372184]



In [32]:
data["token"]

'2uuou6iTpP5tFnIjF3KZjJ4e5sm_58r6P5rmD9ohT8WYS24bP'

In [33]:
!ngrok authtoken data["token"]

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [34]:
import os
import multiprocessing
 
pool = multiprocessing.Pool(processes = 10)
results_of_processes = [pool.apply_async(os.system, args=(cmd, ), callback = None )
                        for cmd in [
                        f"tensorboard --logdir /kaggle/working/logs --load_fast=false --host 0.0.0.0 --port 6006 &",
                        "/usr/local/bin/ngrok http 6006 &"
                        ]]

In [35]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://191a-34-40-140-38.ngrok-free.app


In [25]:
# !load_ext tensorboard.notebook
# !tensorboard --logdir /kaggle/working/weights/logs

In [26]:
# !pip uninstall -y tensorboard tb-nightly &&
# !pip install tb-nightly  # must have at least tb-nightly==2.5.0a20210316

In [27]:
# %load_ext tensorboard
# %tensorboard --logdir /kaggle/working/weights/logs

In [28]:
# %reload_ext tensorboard